We replicate exactly the benchmark models to allow for the best direct comparison

In [1]:
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

Using TensorFlow backend.


In [2]:
base_vgg16 = VGG16(weights='imagenet')

# Second Stage 1: VGG16 with replaced last layer

In [3]:
from keras.models import Model
from keras.layers import Dense

Replace the layer

In [4]:
final_layer = Dense(22, activation='softmax')(base_vgg16.get_layer('fc2').output)
second_stage_last_layer = Model(base_vgg16.input, final_layer)

In [5]:
second_stage_last_layer.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

Make all other layers not trainable

In [6]:
for layer in second_stage_last_layer.layers[:-1]:
    layer.trainable = False

In [7]:
from keras.metrics import top_k_categorical_accuracy
def top_1_error(y_true, y_pred):
    return 1-top_k_categorical_accuracy(y_true, y_pred, k=1)

def top_3_error(y_true, y_pred):
    return 1-top_k_categorical_accuracy(y_true, y_pred, k=3)

In [8]:
second_stage_last_layer.compile(loss='categorical_crossentropy', optimizer='adam',
                                metrics=[top_1_error, top_3_error]) 

Data generators for preprocessed data

In [9]:
def preprocess_img(img):
    img = np.expand_dims(img, axis=0)
    return preprocess_input(img)

datagen_train = ImageDataGenerator(preprocessing_function=preprocess_img,
                                   horizontal_flip=True)

def generator_train(batch_size):
    return datagen_train.flow_from_directory(
        './data/f_rcnn_second_stage/train', target_size=(224, 224),
        batch_size=batch_size, class_mode='categorical')

datagen_val = ImageDataGenerator(preprocessing_function=preprocess_img)

def generator_val(batch_size, shuffle=True):
    return datagen_val.flow_from_directory(
        './data/f_rcnn_second_stage/val', target_size=(224, 224),
        batch_size=batch_size, class_mode='categorical', 
        shuffle=shuffle)

In [10]:
import pandas as pd
image_level_statistics = pd.read_csv('./results/data_analysis/image_level_statistics.csv')
n_train_images = (image_level_statistics['sample'] == 'train').sum()
n_val_images = (image_level_statistics['sample'] == 'val').sum()

Some callbacks. We save the model with the lowest validation loss, stop if there was no decrease in 20 epochs, and track the history of losses and errors.

In [11]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, LambdaCallback
checkpointer = ModelCheckpoint(filepath='./saved_models/f_rcnn/second_stage/weights_vgg16_top_only.hdf5', 
                               verbose=1, save_best_only=True)
stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1, mode='auto')

In [12]:
history_last_layer = list()
def get_history(history_dict):
    historian = LambdaCallback(
        on_epoch_end=lambda epoch, logs:  history_dict.append(
            {'loss': logs['loss'], 
             'top_1_error': logs['top_1_error'],
             'top_3_error': logs['top_3_error'], 
             'val_loss': logs['val_loss'],  
             'val_top_1_error': logs['val_top_1_error'], 
             'val_top_3_error': logs['val_top_3_error']}
        )
    )
    return historian

In [13]:
epochs = 100

In [14]:
batch_size = 32

In [15]:
second_stage_last_layer.fit_generator(generator_train(batch_size), 
                                   steps_per_epoch=n_train_images/(5*batch_size),
                                   epochs=epochs, 
                                   callbacks = [checkpointer, stopper, 
                                                get_history(history_last_layer)], 
                                   validation_data=generator_val(batch_size), 
                                   validation_steps=n_val_images/(5*batch_size), 
                                   verbose=2)

Found 12990 images belonging to 22 classes.
Found 4319 images belonging to 22 classes.
Epoch 1/100
Epoch 00000: val_loss improved from inf to 2.11749, saving model to ./saved_models/f_rcnn/second_stage/weights_vgg16_top_only.hdf5
32s - loss: 2.6063 - top_1_error: 0.6646 - top_3_error: 0.3982 - val_loss: 2.1175 - val_top_1_error: 0.5706 - val_top_3_error: 0.2685
Epoch 2/100
Epoch 00001: val_loss improved from 2.11749 to 2.11080, saving model to ./saved_models/f_rcnn/second_stage/weights_vgg16_top_only.hdf5
31s - loss: 2.0760 - top_1_error: 0.5499 - top_3_error: 0.2923 - val_loss: 2.1108 - val_top_1_error: 0.5081 - val_top_3_error: 0.2894
Epoch 3/100
Epoch 00002: val_loss improved from 2.11080 to 2.03129, saving model to ./saved_models/f_rcnn/second_stage/weights_vgg16_top_only.hdf5
31s - loss: 2.0648 - top_1_error: 0.5343 - top_3_error: 0.2645 - val_loss: 2.0313 - val_top_1_error: 0.5093 - val_top_3_error: 0.2847
Epoch 4/100
Epoch 00003: val_loss improved from 2.03129 to 1.95914, saving

In [16]:
second_stage_last_layer.load_weights('./saved_models/f_rcnn/second_stage/weights_vgg16_top_only.hdf5')
performance_second_stage_last_layer = second_stage_last_layer.evaluate_generator(
    generator_val(batch_size, shuffle=False), steps = n_val_images/batch_size)
print('Validation loss: {:2f}'.format(performance_second_stage_last_layer[0]))
print('Validation top-1-error rate: {:.2f}%'.format(100*performance_second_stage_last_layer[1]))
print('Validation top-3-error rate: {:.2f}%'.format(100*performance_second_stage_last_layer[2]))

Found 4319 images belonging to 22 classes.
Validation loss: 1.860133
Validation top-1-error rate: 47.67%
Validation top-3-error rate: 22.18%


Save the results

In [17]:
pd.DataFrame(history_last_layer).to_csv('./results/f_rcnn/second_stage/history_last_layer.csv')

# Second Stage 2: VGG16 with replaced dense layers


In [18]:
from keras.layers import Dropout, GlobalAveragePooling2D

In [19]:
dense = GlobalAveragePooling2D()(base_vgg16.get_layer('block5_pool').output)
dense = Dense(2048, activation='relu', name='fc1')(dense)
dense = Dropout(0.3)(dense)
dense = Dense(2048, activation='relu', name='fc2')(dense)
dense = Dropout(0.3)(dense)
dense = Dense(22, activation='softmax', name='prediction')(dense)

In [20]:
second_stage_all_dense_layers = Model(base_vgg16.input, dense)

In [21]:
second_stage_all_dense_layers.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [22]:
print('Which layers are trainable?')
for layer in second_stage_all_dense_layers.layers:
    print('{}: {}'.format(layer.name, layer.trainable))

Which layers are trainable?
input_1: False
block1_conv1: False
block1_conv2: False
block1_pool: False
block2_conv1: False
block2_conv2: False
block2_pool: False
block3_conv1: False
block3_conv2: False
block3_conv3: False
block3_pool: False
block4_conv1: False
block4_conv2: False
block4_conv3: False
block4_pool: False
block5_conv1: False
block5_conv2: False
block5_conv3: False
block5_pool: False
global_average_pooling2d_1: True
fc1: True
dropout_1: True
fc2: True
dropout_2: True
prediction: True


In [23]:
second_stage_all_dense_layers.compile(loss='categorical_crossentropy', optimizer='adam',
                                   metrics=[top_1_error, top_3_error])

In [24]:
checkpointer = ModelCheckpoint(filepath='./saved_models/f_rcnn/second_stage/weights_vgg16_dense.hdf5', 
                               verbose=1, save_best_only=True)

In [25]:
history_all_dense_layers = list()

In [26]:
batch_size=25

In [27]:
second_stage_all_dense_layers.fit_generator(generator_train(batch_size), 
                                         steps_per_epoch=n_train_images/(5*batch_size),
                                         epochs=epochs, 
                                         callbacks = [checkpointer, stopper, 
                                                      get_history(history_all_dense_layers)], 
                                         validation_data=generator_val(batch_size), 
                                         validation_steps=n_val_images/(5*batch_size), 
                                         verbose=2)

Found 12990 images belonging to 22 classes.
Found 4319 images belonging to 22 classes.
Epoch 1/100
Epoch 00000: val_loss improved from inf to 1.87255, saving model to ./saved_models/f_rcnn/second_stage/weights_vgg16_dense.hdf5
36s - loss: 4.9774 - top_1_error: 0.7496 - top_3_error: 0.5273 - val_loss: 1.8725 - val_top_1_error: 0.5771 - val_top_3_error: 0.2926
Epoch 2/100
Epoch 00001: val_loss improved from 1.87255 to 1.85063, saving model to ./saved_models/f_rcnn/second_stage/weights_vgg16_dense.hdf5
35s - loss: 2.0243 - top_1_error: 0.6046 - top_3_error: 0.3377 - val_loss: 1.8506 - val_top_1_error: 0.5611 - val_top_3_error: 0.2949
Epoch 3/100
Epoch 00002: val_loss improved from 1.85063 to 1.81115, saving model to ./saved_models/f_rcnn/second_stage/weights_vgg16_dense.hdf5
35s - loss: 1.9835 - top_1_error: 0.5965 - top_3_error: 0.3246 - val_loss: 1.8111 - val_top_1_error: 0.5497 - val_top_3_error: 0.2777
Epoch 4/100
Epoch 00003: val_loss improved from 1.81115 to 1.59850, saving model to

In [28]:
second_stage_all_dense_layers.load_weights('./saved_models/f_rcnn/second_stage/weights_vgg16_dense.hdf5')
performance_second_stage_all_dense_layers = second_stage_all_dense_layers.evaluate_generator(
    generator_val(batch_size, shuffle=False), steps = n_val_images/batch_size)
print('Validation loss: {:2f}'.format(performance_second_stage_all_dense_layers[0]))
print('Validation top-1-error rate: {:.2f}%'.format(100*performance_second_stage_all_dense_layers[1]))
print('Validation top-3-error rate: {:.2f}%'.format(100*performance_second_stage_all_dense_layers[2]))

Found 4319 images belonging to 22 classes.
Validation loss: 1.483330
Validation top-1-error rate: 43.11%
Validation top-3-error rate: 19.40%


Save results

In [29]:
pd.DataFrame(history_all_dense_layers).to_csv('./results/f_rcnn/second_stage/history_all_dense_layers.csv')

# Second Stage 3: VGG16 with replaced dense layers and trained convolutional block

In [30]:
second_stage_dense_and_conv = second_stage_all_dense_layers

In [31]:
for layer in second_stage_dense_and_conv.layers[15:]:
    layer.trainable = True

In [32]:
print('Which layers are trainable?')
for layer in second_stage_dense_and_conv.layers:
    print('{}: {}'.format(layer.name, layer.trainable))

Which layers are trainable?
input_1: False
block1_conv1: False
block1_conv2: False
block1_pool: False
block2_conv1: False
block2_conv2: False
block2_pool: False
block3_conv1: False
block3_conv2: False
block3_conv3: False
block3_pool: False
block4_conv1: False
block4_conv2: False
block4_conv3: False
block4_pool: False
block5_conv1: True
block5_conv2: True
block5_conv3: True
block5_pool: True
global_average_pooling2d_1: True
fc1: True
dropout_1: True
fc2: True
dropout_2: True
prediction: True


We use a slower learning rate here for fine-tuning

In [33]:
from keras.optimizers import SGD
sgd = SGD(lr=10e-4, momentum=0.9, nesterov=True)

In [34]:
second_stage_dense_and_conv.compile(loss='categorical_crossentropy', optimizer=sgd,
                                 metrics=[top_1_error, top_3_error])

In [35]:
checkpointer = ModelCheckpoint(filepath='./saved_models/f_rcnn/second_stage/weights_vgg16_conv_and_dense.hdf5', 
                               verbose=1, save_best_only=True)

In [36]:
history_dense_and_conv = list()

In [37]:
batch_size=20

In [38]:
second_stage_dense_and_conv.fit_generator(generator_train(batch_size), 
                                       steps_per_epoch=n_train_images/(5*batch_size),
                                       epochs=epochs, 
                                       callbacks = [checkpointer, stopper, 
                                                    get_history(history_dense_and_conv)], 
                                       validation_data=generator_val(batch_size), 
                                       validation_steps=n_val_images/(5*batch_size), 
                                       verbose=2)

Found 12990 images belonging to 22 classes.
Found 4319 images belonging to 22 classes.
Epoch 1/100
Epoch 00000: val_loss improved from inf to 1.85537, saving model to ./saved_models/f_rcnn/second_stage/weights_vgg16_conv_and_dense.hdf5
42s - loss: 2.0202 - top_1_error: 0.6138 - top_3_error: 0.3419 - val_loss: 1.8554 - val_top_1_error: 0.5500 - val_top_3_error: 0.2636
Epoch 2/100
Epoch 00001: val_loss improved from 1.85537 to 1.77311, saving model to ./saved_models/f_rcnn/second_stage/weights_vgg16_conv_and_dense.hdf5
41s - loss: 1.7809 - top_1_error: 0.5454 - top_3_error: 0.2754 - val_loss: 1.7731 - val_top_1_error: 0.5341 - val_top_3_error: 0.2500
Epoch 3/100
Epoch 00002: val_loss improved from 1.77311 to 1.70867, saving model to ./saved_models/f_rcnn/second_stage/weights_vgg16_conv_and_dense.hdf5
41s - loss: 1.6679 - top_1_error: 0.5008 - top_3_error: 0.2192 - val_loss: 1.7087 - val_top_1_error: 0.5114 - val_top_3_error: 0.2511
Epoch 4/100
Epoch 00003: val_loss improved from 1.70867 

In [39]:
second_stage_dense_and_conv.load_weights('./saved_models/f_rcnn/second_stage/weights_vgg16_conv_and_dense.hdf5')
performance_second_stage_conv_and_dense_layers = second_stage_dense_and_conv.evaluate_generator(
    generator_val(batch_size, shuffle=False), steps = n_val_images/batch_size)
print('Validation loss: {:2f}'.format(performance_second_stage_conv_and_dense_layers[0]))
print('Validation top-1-error rate: {:.2f}%'.format(100*performance_second_stage_conv_and_dense_layers[1]))
print('Validation top-3-error rate: {:.2f}%'.format(100*performance_second_stage_conv_and_dense_layers[2]))

Found 4319 images belonging to 22 classes.
Validation loss: 1.341645
Validation top-1-error rate: 38.64%
Validation top-3-error rate: 16.46%


Save results

In [40]:
pd.DataFrame(history_dense_and_conv).to_csv('./results/f_rcnn/second_stage/history_dense_and_conv.csv')